# Embedding 3D Data with M3D-CLIP in Julia using PythonCall
The high-level overview of this notebooks is that we will be loading in CT data and obtaining an embedding to be used in other models with the `M3D-CLIP` foundation model.

To do so, we need to do the following:
1. Create a Python Environment for PythonCall to properly support the `M3D` model as well as the `CLIP` tokenizer
2. Use PythonCall to load in the necessary Python modules to interact with `M3D`
3. Make use of the embedded data!

## 0. Set Up Our Julia Environment
The only packages we will need are `CondaPkg.jl` to manage our Python environemtn and `PythonCall.jl` to interface between Python and Julia.

In [1]:
using Pkg
Pkg.activate(".")
Pkg.add("CondaPkg")
Pkg.add("PythonCall")
Pkg.add("JLD2")

  Activating new project at `/content`
    Updating registry at `~/.julia/registries/General.toml`
   Resolving package versions...
   Installed micromamba_jll ─ v1.5.8+0
   Installed pixi_jll ─────── v0.41.3+0
   Installed MicroMamba ───── v0.1.14
   Installed CondaPkg ─────── v0.2.28
   Installed Pidfile ──────── v1.3.0
   Installed JSON3 ────────── v1.14.2
   Installed StructTypes ──── v1.11.0
    Updating `/content/Project.toml`
  [992eb4ea] + CondaPkg v0.2.28
    Updating `/content/Manifest.toml`
  [992eb4ea] + CondaPkg v0.2.28
  [692b3bcd] + JLLWrappers v1.7.0
  [0f8b85d8] + JSON3 v1.14.2
  [0b3b1443] + MicroMamba v0.1.14
  [69de0a69] + Parsers v2.8.3
  [fa939f87] + Pidfile v1.3.0
⌅ [aea7be01] + PrecompileTools v1.2.1
  [21216c6a] + Preferences v1.4.3
  [6c6a2e73] + Scratch v1.2.1
  [856f2bd8] + StructTypes v1.11.0
  [f8abcde7] + micromamba_jll v1.5.8+0
  [4d7b5844] + pixi_jll v0.41.3+0
  [0dad84c5] + ArgTools v1.1.1
  [56f22d72] + Artifacts
  [2a0f44e3] + Base64
  [ade2ca70] + D

## 1. Create Our Python Environment
To create our Python environment, using `CondaPkg.jl`, we will do the following actions:
1. Install python 3.11 (we need that exact version for a few Python libraries to function)
2. Read in a `requirements.txt` file to extract the necessary Python libraries and their specific version numbers
3. Use `pip` to install the libraries

In [2]:
import CondaPkg

### Installing Python 3.11

In [3]:
CondaPkg.add("python"; version=">3.10,<3.12")

    CondaPkg Found dependencies: /content/CondaPkg.toml
    CondaPkg Found dependencies: /root/.julia/packages/PythonCall/WMWY0/CondaPkg.toml
    CondaPkg Found dependencies: /root/.julia/packages/Reactant/OgayD/CondaPkg.toml
    CondaPkg Resolving changes
             + jax (pip)
             + libstdcxx-ng
             + python
             + uv
    CondaPkg Initialising pixi
             │ /root/.julia/artifacts/cefba4912c2b400756d043a2563ef77a0088866b/bin/pixi
             │ init
             │ --format pixi
             └ /content/.CondaPkg
✔ Created /content/.CondaPkg/pixi.toml
    CondaPkg Wrote /content/.CondaPkg/pixi.toml
             │ [dependencies]
             │ uv = ">=0.4"
             │ libstdcxx-ng = ">=3.4,<13.0"
             │ 
             │     [dependencies.python]
             │     channel = "conda-forge"
             │     build = "*cpython*"
             │     version = ">=3.8,<4, >3.10,<3.12"
             │ 
             │ [project]
             │ name = ".Co

### Parsing through the `requirements.txt` File
Directly using a `requirements.txt` file is not yet supported in `CondaPkg.jl`. However, we can use basic file reading operations in `Base` to get what we want

In [1]:
function parse_pip_requirements(path)
    packages = []
    versions = []
    open(path) do file
        for line in eachline(file)
            # Skip empty lines and comments
            if isempty(line) || line[1] == '#'
                continue
            end

            # Split the line into package and version
            package, version = split(line, "==")
            version = "==" * version
            push!(packages, package)
            push!(versions, version)
        end
    end
    return packages, versions
end

packages, versions = parse_pip_requirements("requirements.txt")
println(packages)
println(versions)

Any["deepspeed", "einops", "evaluate", "matplotlib", "monai", "nibabel", "numpy", "opencv_python", "pandas", "peft", "Pillow", "pycocotools", "Requests", "rouge", "safetensors", "scipy", "simple_slice_viewer", "SimpleITK", "torch", "torchvision", "tqdm", "transformers", "tweepy"]
Any["==0.13.4", "==0.8.0", "==0.4.1", "==3.8.4", "==1.2.0", "==5.2.1", "==1.26.4", "==4.9.0.80", "==2.2.2", "==0.8.2", "==10.3.0", "==2.0.7", "==2.31.0", "==1.0.1", "==0.4.3", "==1.13.0", "==0.97", "==2.3.1", "==2.6.0", "==0.21.0", "==4.66.2", "==4.39.1", "==4.14.0"]


### Installing the Python Libraries
We can install each library individually with `pip` through `CondaPkg.add_pip(package_name; version=package_version)`

In [5]:
for i in 1:length(packages)
    CondaPkg.add_pip(packages[i]; version=versions[i])
end

    CondaPkg Found dependencies: /content/CondaPkg.toml
    CondaPkg Found dependencies: /root/.julia/packages/PythonCall/WMWY0/CondaPkg.toml
    CondaPkg Found dependencies: /root/.julia/packages/Reactant/OgayD/CondaPkg.toml
    CondaPkg Resolving changes
             + deepspeed (pip)
    CondaPkg Initialising pixi
             │ /root/.julia/artifacts/cefba4912c2b400756d043a2563ef77a0088866b/bin/pixi
             │ init
             │ --format pixi
             └ /content/.CondaPkg
✔ Created /content/.CondaPkg/pixi.toml
    CondaPkg Wrote /content/.CondaPkg/pixi.toml
             │ [dependencies]
             │ uv = ">=0.4"
             │ libstdcxx-ng = ">=3.4,<13.0"
             │ 
             │     [dependencies.python]
             │     channel = "conda-forge"
             │     build = "*cpython*"
             │     version = ">=3.8,<4, >3.10,<3.12"
             │ 
             │ [project]
             │ name = ".CondaPkg"
             │ platforms = ["linux-64"]
             │

### Verify That we Have the Correct Python Version and Libraries Installed

In [6]:
CondaPkg.status()

CondaPkg Status /content/CondaPkg.toml
Environment
  /content/.CondaPkg/.pixi/envs/default
Packages
  python v3.11.12 (>3.10,<3.12)
Pip packages
  deepspeed v0.13.4 (==0.13.4)
  einops v0.8.0 (==0.8.0)
  evaluate v0.4.1 (==0.4.1)
  matplotlib v3.8.4 (==3.8.4)
  monai v1.2.0 (==1.2.0)
  nibabel v5.2.1 (==5.2.1)
  numpy v1.26.4 (==1.26.4)
  opencv-python v4.9.0.80 (==4.9.0.80)
  pandas v2.2.2 (==2.2.2)
  peft v0.8.2 (==0.8.2)
  pillow v10.3.0 (==10.3.0)
  pycocotools v2.0.7 (==2.0.7)
  requests v2.31.0 (==2.31.0)
  rouge v1.0.1 (==1.0.1)
  safetensors v0.4.3 (==0.4.3)
  scipy v1.13.0 (==1.13.0)
  simple-slice-viewer v0.97 (==0.97)
  simpleitk v2.3.1 (==2.3.1)
  torch v2.6.0 (==2.6.0)
  torchvision v0.21.0 (==0.21.0)
  tqdm v4.66.2 (==4.66.2)
  transformers v4.39.1 (==4.39.1)
  tweepy v4.14.0 (==4.14.0)


## 2. Using `PythonCall.jl` to Load in the Model
Using `PythonCall.jl`, we simply load in the `M3D-CLIP` model and its tokenizer from Hugging Face, and we can interact with the Python model in a very Julian way.

In [1]:
import PythonCall
torch = PythonCall.pyimport("torch")
np = PythonCall.pyimport("numpy")
AutoTokenizer = PythonCall.pyimport("transformers").AutoTokenizer
AutoModel = PythonCall.pyimport("transformers").AutoModel

tokenizer = AutoTokenizer.from_pretrained("GoodBaiBai88/M3D-CLIP", model_max_length=512, padding_side="right", use_fast=false)
model = AutoModel.from_pretrained("GoodBaiBai88/M3D-CLIP", trust_remote_code=true)

/home/hasith/Documents/pumas-ai/3d-models/.CondaPkg/.pixi/envs/default/lib/python3.11/site-packages/huggingface_hub/file_download.py:896: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/home/hasith/Documents/pumas-ai/3d-models/.CondaPkg/.pixi/envs/default/lib/python3.11/site-packages/huggingface_hub/file_download.py:896: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Python:
M3DCLIP(
  (vision_encoder): ViT(
    (patch_embedding): PatchEmbeddingBlock(
      (patch_embeddings): Sequential(
        (0): Rearrange('b c (h p1) (w p2) (d p3) -> b (h w d) (p1 p2 p3 c)', p1…
        (1): Linear(in_features=1024, out_features=768, bias=True)
      )
      (dropout): Dropout(p=0, inplace=False)
    )
    (blocks): ModuleList(
    ... 57 more lines ...
    )
    (pooler): BertPooler(
      (dense): Linear(in_features=768, out_features=768, bias=True)
      (activation): Tanh()
    )
  )
  (mm_vision_proj): Linear(in_features=768, out_features=768, bias=True)
  (mm_language_proj): Linear(in_features=768, out_features=768, bias=True)
)

## 3. Obtaining the Embeddings
To get the embeddings, all we have to do is use the model's built in encoder which will embed the image and return its embedding.

In [ ]:
using JLD2

image = JLD2.load("data/example.jld2", "jl_image")
torch_image = torch.tensor(PythonCall.pyrowlist(image)).unsqueeze(0)
torch_image.shape

torch_encoded_image = model.encode_image(torch_image)
torch_image.shape, torch_encoded_image.shape

(<py torch.Size([1, 1, 32, 256, 256])>, <py torch.Size([1, 2049, 768])>)

In [40]:
encoded_image = PythonCall.pyconvert(Array, torch_encoded_image.detach().numpy())

1×2049×768 Array{Float32, 3}:
[:, :, 1] =
 0.0206281  -0.006389  -0.00490209  -0.0278195  …  -0.00684712  -0.00559487

[:, :, 2] =
 0.0736189  0.069139  0.0704843  0.057757  …  0.0697616  0.0697715  0.0685673

[:, :, 3] =
 0.00992233  0.00230995  -0.000363045  0.0486071  …  0.000932447  0.00255924

;;; … 

[:, :, 766] =
 0.0498103  0.0209607  0.0230589  …  0.0226209  0.0219514  0.0209882

[:, :, 767] =
 0.00530913  0.0151308  0.0169863  …  0.0146035  0.0167086  0.0118581

[:, :, 768] =
 -0.0554023  -0.0689056  -0.0704629  …  -0.0680503  -0.0677214  -0.0672402